# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice,randint
import os
import joblib




## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-capstone1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135180
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-135180


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
cpu_cluster_name = "cpuacluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
    
except ComputeTargetException:
    
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("New cluster is created")

Found existing cluster


In [10]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        'C':uniform(0,1),
        "max_iter":randint(150)
    })

if "training" not in os.listdir():
    os.mkdir("./training")
#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory='./',  
              compute_target=cpu_cluster,
              entry_script='train.py', 
              vm_size = 'STANDARD_D12_V2', 
            )
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)

In [11]:
#TODO: Submit your experiment
hdr=exp.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('parameter values:',parameter_values)
print('\n Best run \n',best_run)

print('The Id for best run is:', best_run.id)
print('The Accuracy: is', best_run_metrics['Accuracy'])

parameter values: ['--C', '0.6361917497447508', '--max_iter', '92']

 Best run 
 Run(Experiment: udacity-capstone1,
Id: HD_7e531409-d5db-421d-b4bf-861901d8d8b1_2,
Type: azureml.scriptrun,
Status: Completed)
The Id for best run is: HD_7e531409-d5db-421d-b4bf-861901d8d8b1_2
The Accuracy: is 0.9032258064516129


In [14]:
#get files
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_b66aba6db7dd5e45a82584b944f60a04f35629291cb306ffcfd0450da06a2a2e_d.txt',
 'azureml-logs/65_job_prep-tvmps_b66aba6db7dd5e45a82584b944f60a04f35629291cb306ffcfd0450da06a2a2e_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_b66aba6db7dd5e45a82584b944f60a04f35629291cb306ffcfd0450da06a2a2e_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/100_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [15]:
#register model
the_best_model = best_run.register_model(model_name='best-model.joblib', model_path='./')

In [16]:
#Display the best model
the_best_model

Model(workspace=Workspace.create(name='quick-starts-ws-135180', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-135180'), name=best-model.joblib, id=best-model.joblib:1, version=1, tags={}, properties={})

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service